Go directly to:
- [**Start page**](https://github.com/coconeuro/remeta/)
- [**Installation**](https://github.com/coconeuro/remeta/blob/main/INSTALL.md)
- [**Basic Usage**](https://github.com/coconeuro/remeta/blob/main/demo/basic_usage.ipynb)
- [**Common use cases**](https://github.com/coconeuro/remeta/blob/main/demo/common_use_cases.ipynb)
- [**Group estimation and priors** (this page)](https://github.com/coconeuro/remeta/blob/main/demo/group_estimation_priors.ipynb)

## Group estimation and priors

Often, data from single participants is not sufficient for precise parameter estimation. In this case, ReMeta offers three different options to enrich parameter estimation either by group-level or prior information.

### Group estimation (fixed effect)

To use group-level information we need to pass 1) a 2d data to ReMeta (nsubjects x nsamples) and 2) specify the `group` attribute for parameters.

The `fit` method of ReMeta accepts data as either 1d arrays (single participant) or as 2d arrays (group data). To showcase, we simulate type 1 data for 4 participants:


In [24]:
import remeta
import numpy as np
np.random.seed(0)
cfg = remeta.Configuration()
cfg.param_type1_nonlinear_gain.enable = 1
cfg.skip_type2 = True
params_true = dict(
    type1_noise=0.5,
    type1_bias=0,
    type1_nonlinear_gain=0.5
)
cfg.true_params = params_true
data = remeta.simulate(nsubjects=4, nsamples=2000, params=params_true, cfg=cfg)

----------------------------------
..Generative parameters:
    Type 1 noise distribution: normal
    type1_noise: 0.5
    type1_bias: 0
    type1_nonlinear_gain: 0.5
..Descriptive statistics:
    No. subjects: 4
    No. samples: 2000
    Accuracy: 84.4% correct
    d': 2.0
    Choice bias: -0.4%
----------------------------------


We simulated data with a slight non-linearity (`type1_param_nonlinear_gain=0.5`). Non-linearities in the encoding of the stimulus intensity are notoriously sample-intensive and thus estimates for single participants are often all over the place even if fitted with the same ground truth model. This is exactly what we find, even though each participant has 2000 trials:

In [25]:
rem = remeta.ReMeta(cfg=cfg)
rem.fit(data.stimuli, data.choices, data.confidence)
result = rem.summary()

Dataset characteristics:
    No. subjects: 4
    No. samples: [2000, 2000, 2000, 2000]
    Accuracy: 84.4% correct
    d': 2.023
    Choice bias: -0.4%

+++ Type 1 level +++
  Subject-level estimation (MLE)
     Subject 1 / 4
     Subject 2 / 4
     Subject 3 / 4
     Subject 4 / 4
    .. finished (1.1 secs).
  Final report
    Subject 1 / 4
        Parameters estimates (subject-level fit)
            [subject] type1_noise: 0.536 ± 0.056 (true: 0.500)
            [subject] type1_nonlinear_gain: 0.505 ± 0.373 (true: 0.500)
            [subject] type1_bias: -0.023 ± 0.020 (true: 0.000)
        [subject] Log-likelihood: -748.56 (per sample: -0.3743)
        [subject] Fitting time: 0.23 secs
        Log-likelihood using true params: -1177.70 (per sample: -0.5888)
    Subject 2 / 4
        Parameters estimates (subject-level fit)
            [subject] type1_noise: 0.472 ± 0.039 (true: 0.500)
            [subject] type1_nonlinear_gain: 0.107 ± 0.264 (true: 0.500)
            [subject] type1_

In case of a group-level fit, the result returned by the `summary()` method is a list of length `nsubjects`. We can print the final parameter estimates more cleanly as follows:

In [26]:
for s in range(result.nsubjects):
    print(f'Subject {s}')
    for k, v in result.type1.params[s].items():
        print(f'\t{k}: {v:.3f}')

Subject 0
	type1_noise: 0.536
	type1_nonlinear_gain: 0.505
	type1_bias: -0.023
Subject 1
	type1_noise: 0.472
	type1_nonlinear_gain: 0.107
	type1_bias: -0.026
Subject 2
	type1_noise: 0.482
	type1_nonlinear_gain: 0.569
	type1_bias: 0.007
Subject 3
	type1_noise: 0.508
	type1_nonlinear_gain: 0.386
	type1_bias: 0.005


The fitted parameters for `type1_nonlinear_gain` vary strongly around the true value of `0.5`. Yet, this is no fitting error (the empirical negative log-likelihood is always lower than one for the true parameters), but caused by the fact that non-linearities are very hard to infer from binary choice data. Not impossible, but it takes lots of samples to get anywhere near an acceptable level f precision.

One option to tackle this is to fit a single non-linearity parameter to the entire group. To specify, we set the `group` attribute of the parameter to `'fixed'`:

In [27]:
cfg.param_type1_nonlinear_gain.group = 'fixed'

We fit the model as usual:

In [28]:
rem = remeta.ReMeta(cfg=cfg)
rem.fit(data.stimuli, data.choices, data.confidence)
result = rem.summary()

Dataset characteristics:
    No. subjects: 4
    No. samples: [2000, 2000, 2000, 2000]
    Accuracy: 84.4% correct
    d': 2.023
    Choice bias: -0.4%

+++ Type 1 level +++
  Subject-level estimation (MLE)
     Subject 1 / 4
     Subject 2 / 4
     Subject 3 / 4
     Subject 4 / 4
    .. finished (1.1 secs).

  Group-level optimization (MLE / MAP)
        [21:03:55] Iteration 1 / 30
        [21:03:56] Iteration 11 / 30
        [21:03:56] Iteration 21 / 30
    .. finished (1.3 secs).
  Final report
    Subject 1 / 4
        Parameters estimates (subject-level fit)
            [subject] type1_noise: 0.536 ± 0.056 (true: 0.500)
            [subject] type1_nonlinear_gain: 0.505 ± 0.373 (true: 0.500)
            [subject] type1_bias: -0.023 ± 0.020 (true: 0.000)
        [subject] Log-likelihood: -748.56 (per sample: -0.3743)
        [subject] Fitting time: 0.23 secs
        Parameters estimates (group-level fit)
            [subject] type1_noise: 0.518 ± 0.019 (true: 0.500)
            [gr

Now, the parameter `type1_nonlinear_gain` is fitted to the entire group dataset and thus the parameter is identical for each participant. The final estimate of `0.492` much closer to the ground truth value of `0.5`. We once again print the final parameter more cleanly:

In [29]:
for s in range(result.nsubjects):
    print(f'Subject {s}')
    for k, v in result.type1.params[s].items():
        print(f'\t{k}: {v:.3f}')

Subject 0
	type1_noise: 0.518
	type1_nonlinear_gain: 0.381
	type1_bias: -0.023
Subject 1
	type1_noise: 0.509
	type1_nonlinear_gain: 0.381
	type1_bias: -0.028
Subject 2
	type1_noise: 0.460
	type1_nonlinear_gain: 0.381
	type1_bias: 0.007
Subject 3
	type1_noise: 0.507
	type1_nonlinear_gain: 0.381
	type1_bias: 0.005


Note that even though parameter recovery improved, the log-likelihood of this group fit is worse (i.e. lower) than the single-subject fit:

In [30]:
for s in range(result.nsubjects):
    print(f'Subject {s}')
    print(f'\tnegll(subject fit): {result.type1.subject.loglik[s]:.3f}')
    print(f'\tnegll(group fit): {result.type1.group.loglik[s]:.3f}')

Subject 0
	negll(subject fit): -748.555
	negll(group fit): -748.615
Subject 1
	negll(subject fit): -735.225
	negll(group fit): -735.602
Subject 2
	negll(subject fit): -673.607
	negll(group fit): -673.740
Subject 3
	negll(subject fit): -733.734
	negll(group fit): -733.734


Yet, in this case this effectively means that the model is not overfit to random peculiarities of each subject's data and better fits the broad trends in the group data.

### Group estimation (random effect)

Another possibility is to treat a parameter as a random effect. When modeled as a random effect, each subject is fitted to its own data, but the parameter estimate is informed / regularized by an estimate of the parameter's group distribution. If you assume that there is plausibly variability between participants (and there typically is), modeling parameters as random effects might be the better choice.

In [31]:
cfg.param_type1_nonlinear_gain.group = 'random'
rem = remeta.ReMeta(cfg=cfg)
rem.fit(data.stimuli, data.choices, data.confidence)
result = rem.summary()

Dataset characteristics:
    No. subjects: 4
    No. samples: [2000, 2000, 2000, 2000]
    Accuracy: 84.4% correct
    d': 2.023
    Choice bias: -0.4%

+++ Type 1 level +++
  Subject-level estimation (MLE)
     Subject 1 / 4
     Subject 2 / 4
     Subject 3 / 4
     Subject 4 / 4
    .. finished (1.1 secs).

  Group-level optimization (MLE / MAP)
        [21:03:58] Iteration 1 / 30 (Convergence: 0.00012781)
        [21:03:59] Iteration 11 / 30 (Convergence: 0.00001832)
        [21:03:59] Iteration 21 / 30 (Convergence: 0.00000382)
    .. finished (1.6 secs).
  Final report
    Subject 1 / 4
        Parameters estimates (subject-level fit)
            [subject] type1_noise: 0.536 ± 0.056 (true: 0.500)
            [subject] type1_nonlinear_gain: 0.505 ± 0.373 (true: 0.500)
            [subject] type1_bias: -0.023 ± 0.020 (true: 0.000)
        [subject] Log-likelihood: -748.56 (per sample: -0.3743)
        [subject] Fitting time: 0.22 secs
        Parameters estimates (group-level fit)


In the current example, all participant estimates for `type1_nonlinear_gain` are pretty similar, reflecting the fact that the data were in fact generated by the same ground truth model:

In [32]:
for s in range(result.nsubjects):
    print(f'Subject {s}')
    for k, v in result.type1.params[s].items():
        print(f'\t{k}: {v:.6f}')

Subject 0
	type1_noise: 0.519667
	type1_nonlinear_gain: 0.389388
	type1_bias: -0.022693
Subject 1
	type1_noise: 0.506679
	type1_nonlinear_gain: 0.360708
	type1_bias: -0.027539
Subject 2
	type1_noise: 0.461593
	type1_nonlinear_gain: 0.393358
	type1_bias: 0.007012
Subject 3
	type1_noise: 0.506998
	type1_nonlinear_gain: 0.381504
	type1_bias: 0.005352


This observation is matched by an inspection of the population estimate for `type1_nonlinear_gain`:

In [33]:
print(f'Estimated population mean: {result.params_random_effect.mean['type1_nonlinear_gain']:.3f}')

Estimated population mean: 0.384


## Priors

Priors present another way to inform and regularize point estimates of participants. If there is good reason from prior literature or a prior study to assume a prior distribution for a parameter, one can perform Maximum A Posteriori estimation (MAP) instead of Maximum Likelihood estimation (MLE). In Remeta this is possible by specifying the `prior` attribute of a parameter. In the following example, we delete the previous random effect for `type1_param_nonlinear_gain` and specify a prior instead - a tuple of the form (prior_mean, prior_std).

In [34]:
cfg.param_type1_nonlinear_gain.group = None
cfg.param_type1_nonlinear_gain.prior = (0, 0.25)
rem = remeta.ReMeta(cfg=cfg)
rem.fit(data.stimuli, data.choices, data.confidence)
result = rem.summary()

Dataset characteristics:
    No. subjects: 4
    No. samples: [2000, 2000, 2000, 2000]
    Accuracy: 84.4% correct
    d': 2.023
    Choice bias: -0.4%

+++ Type 1 level +++
  Subject-level estimation (MLE)
     Subject 1 / 4
     Subject 2 / 4
     Subject 3 / 4
     Subject 4 / 4
    .. finished (1.1 secs).
  Final report
    Subject 1 / 4
        Parameters estimates (subject-level fit)
            [subject] type1_noise: 0.492 ± 0.032 (true: 0.500)
            [subject+prior=N(0,0.25)] type1_nonlinear_gain: 0.189 ± 0.188 (true: 0.500)
            [subject] type1_bias: -0.021 ± 0.018 (true: 0.000)
        [subject] Log-likelihood: -749.28 (per sample: -0.3746)
        [subject] Fitting time: 0.20 secs
        Log-likelihood using true params: -1177.70 (per sample: -0.5888)
    Subject 2 / 4
        Parameters estimates (subject-level fit)
            [subject] type1_noise: 0.463 ± 0.029 (true: 0.500)
            [subject+prior=N(0,0.25)] type1_nonlinear_gain: 0.047 ± 0.177 (true: 0.5

According to our prior, a null effect for the nonlinearity parameter should be most likely. The precision of the prior is moderate with a standard deviation of `1`. As a consequence, the nonlinearity parameter is biased towards 0 compared to the original estimates without a prior:

In [35]:
for s in range(result.nsubjects):
    print(f'Subject {s}')
    for k, v in result.type1.params[s].items():
        print(f'\t{k}: {v:.3f}')

Subject 0
	type1_noise: 0.492
	type1_nonlinear_gain: 0.189
	type1_bias: -0.021
Subject 1
	type1_noise: 0.463
	type1_nonlinear_gain: 0.047
	type1_bias: -0.025
Subject 2
	type1_noise: 0.440
	type1_nonlinear_gain: 0.208
	type1_bias: 0.007
Subject 3
	type1_noise: 0.476
	type1_nonlinear_gain: 0.154
	type1_bias: 0.005
